# Dashboarding

## Set up
### Set up environment

In [ ]:
# Imports
import json
import os
import logging
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import pycountry

from jupyter_dash import JupyterDash as Dash
from dash import dash_table, dcc, html, Input, Output, State
from pandas.api.types import is_string_dtype

#JupyterDash.infer_jupyter_proxy_config()

### Load data and helpers

In [ ]:
# Helpers
df = pd.read_csv("../data/concatenated/concatenated_all.csv")

countries = {}
for country in pycountry.countries:
    countries[country.name] = country.alpha_3

unabbreviated_countries = [country.strip() for country in df.Country.unique() if country not in countries.keys()]

### Preprocessing

In [ ]:
countries["Belgium-Luxembourg"] = countries["Belgium"]
countries["Congo, Democratic Republic"] = countries["Congo, The Democratic Republic of the"]
countries["Czech Republic"] = countries["Czechia"]
countries["German Federal Republic"] = countries["Germany"]
countries["Ireland, Republic of"] = countries["Ireland"]
countries["Korea (Rep. of)"] = countries["Korea, Republic of"]
countries["Russia"] = countries["Russian Federation"]
countries["Soviet Union"] = countries["Russian Federation"]
countries["Taiwan"] = countries["Taiwan, Province of China"]
countries["USA"] = countries["United States"]
countries["Vietnam"] = countries["Viet Nam"]
countries["Zaire"] = countries["Congo, Democratic Republic"]

# Map country names
df["Country code"] = df["Country"].apply(lambda x: countries.get(x, "Unknown code"))

# Convert years to integers
df["Year"] = df["Year"].astype(int)

# Clear out trailing spaces
df = df[df.columns].apply(lambda c: c.str.strip() if is_string_dtype(c) else c)

# Reordering dataset
column_order = ["Country code", "Country", "Activity", "Commodity", "Sub-commodity", "Year", "Amount", "Measure"]
df = df[column_order]

# Remove rows with missing amount
df = df.dropna(subset=["Amount"], axis=0).reset_index(drop=True)

# 
activity = df["Activity"].unique()[0]
commodity = df["Commodity"].unique()[0]

In [ ]:
initial_activity = np.sort(df["Activity"])[0]
initial_commodity = np.sort(df["Commodity"])[0]
icon_play = "\u25B6"
icon_pause = "\u23F8"

import pandas as pd
import plotly.express as px

app = Dash(__name__)

#df = pd.read_csv('https://plotly.github.io/datasets/country_indicators.csv')

app.layout = html.Div(children=[
    dcc.Interval(id="animate", disabled=True),
    html.Div(children=[
        html.Div(children=[
            dcc.Dropdown(
                df['Activity'].unique(),
                value=initial_activity,
                id='dropdown_activity'
            )
        ],  style={'width': '48%', 'display': 'inline-block'}),
        html.Div(children=[
            dcc.Dropdown(
                df['Commodity'].unique(),
                value=initial_commodity,
                id='dropdown_commodity'
            )
        ], style={'width': '48%', 'float': 'right', 'display': 'inline-block'})
    ]),
    
    dcc.Graph(id='map'),
    dcc.Graph(id='bar'),
    
    html.Div(children=[
        html.Div(children=[
            html.Button(icon_play, id='button_play')
        ], style={'width': '8%', 'display': 'inline-block', "float":"left"}),
    #html.Div(children=[
    #    html.Button(
    #        children=[
    #        dcc.Interval(id='auto-stepper',
    #                     interval=1*1000, # in milliseconds
    #                     n_intervals=0
    #                    )
    #    ], style={'width': '8%', 'display': 'inline-block'}),
        html.Div(children=[
            dcc.Slider(
                min=df['Year'].min(),
                max=df['Year'].max(),
                step=1,
                id='slider_year',
                value=df['Year'].max(),
                marks={str(year): {'label': str(year), 'style': {"transform":"rotate(270deg)"}} for year in df['Year'].unique()}
            )
        ], style={'width': '75%', 'float': 'left', 'display': 'inline-block'})
    ])
])


@app.callback(
    Output("animate", "disabled"),
    Output("button_play", "children"),
    Input('button_play', 'n_clicks'),
    Input("button_play", "children"),
    State("animate", "disabled"),
    prevent_initial_call=True
)
def play_slider(n, button_label, playing):
    if button_label == icon_play:
        button_label = icon_pause
    else:
        button_label = icon_play
    if n:
        return not playing, button_label
    return playing, button_label

@app.callback(
    Output('dropdown_commodity', 'options'),
    Input('dropdown_activity', 'value')
)
def update_commodity_dropdown(dropdown_activity):
    filtered_commodity_df = df[(df["Activity"]==dropdown_activity)]
    filtered_commodity_options = np.sort(filtered_commodity_df["Commodity"].unique())
    return filtered_commodity_options
                               
@app.callback(
    Output('dropdown_activity', 'options'),
    Input('dropdown_commodity', 'value')
)
def update_activity_dropdown(dropdown_commodity):
    filtered_activity_df = df[(df["Commodity"]==dropdown_commodity)]
    filtered_activity_options = np.sort(filtered_activity_df["Activity"].unique())
    return filtered_activity_options

@app.callback(
    Output('slider_year', 'min'),
    Output('slider_year', 'max'),
    Input('dropdown_activity', 'value'),
    Input('dropdown_commodity', 'value')
)
def update_slider(dropdown_activity, dropdown_commodity):
    slider_df = df[(df["Activity"]==dropdown_activity) &
                   (df["Commodity"]==dropdown_commodity)]
    filtered_slider_min = slider_df["Year"].min()
    filtered_slider_max = slider_df["Year"].max()
    return filtered_slider_min, filtered_slider_max

@app.callback(
    Output('map', 'figure'),
    Output('bar', 'figure'),
    Output("slider_year", "value"),
    Input('dropdown_activity', 'value'),
    Input('dropdown_commodity', 'value'),
    Input("animate", "n_intervals"),
    Input('slider_year', 'min'),
    Input('slider_year', 'max'),
    Input('slider_year', 'value')
)
def update_graphs(dropdown_activity, dropdown_commodity,
                 animate, min_year, max_year, slider_year):
    years = range(min_year, max_year+1, 1)
    index = years.index(slider_year)
    index = (index + 1) % len(years)
    year = years[index]
    
    filtered_df = df[(df["Year"]==slider_year) & 
                  (df["Activity"]==dropdown_activity) &
                  (df["Commodity"]==dropdown_commodity)]

    fig_map = px.choropleth(
        filtered_df,
        locations="Country code",
        color="Amount",
        hover_name="Country",
        color_continuous_scale='blugrn',
        height=600,
        title=f"Global {dropdown_activity} of {dropdown_commodity} in {str(slider_year)}"
    )
    
    df_to_bar = filtered_df.groupby(["Year", "Country code"]).sum().sort_values(["Year", "Amount"], ascending=[True, True].head(10))
    
    fig_bar = px.bar(
        df_to_bar,
        x="Amount",
        y=df_to_bar.index.get_level_values(1),
        orientation='h',
        color_continuous_scale="blugrn",
        labels={'y':'Country'}, color="Amount",
        width=900, height=500
    )

    fig_bar.update_layout(margin={"r":10, "t":0,"l":10,"b":0})

    #fig.update_layout(margin={'l': 40, 'b': 40, 't': 10, 'r': 0}, hovermode='closest')
    return fig_map, fig_bar, year

app.run_server(host='0.0.0.0', port=8050, mode="jupyterlab", debug=True)
# 127.0.0.1:8050

from dash import dcc
from dash import html
from jupyter_dash import JupyterDash as Dash
from dash.dependencies import Input, Output, State
import plotly.express as px

import pandas as pd

external_stylesheets = ["https://codepen.io/chriddyp/pen/bWLwgP.css"]

app = Dash(__name__, external_stylesheets=external_stylesheets)
df = pd.read_csv(
    "https://raw.githubusercontent.com/plotly/datasets/master/gapminderDataFiveYear.csv"
)
years = list(set(df["year"]))
years.sort()


def make_fig(year):
    return px.scatter(
        df[df.year == year],
        x="gdpPercap",
        y="lifeExp",
        size="pop",
        color="continent",
        hover_name="country",
        log_x=True,
        size_max=55,
    )


app.layout = html.Div(
    [
        dcc.Interval(id="animate", disabled=True),
        dcc.Graph(id="graph-with-slider", figure=make_fig(1952)),
        dcc.Slider(
            id="year-slider",
            min=df["year"].min(),
            max=df["year"].max(),
            value=df["year"].min(),
            marks={str(year): str(year) for year in df["year"].unique()},
            step=None,
        ),
        html.Button("Play", id="play"),
    ]
)


@app.callback(
    Output("graph-with-slider", "figure"),
    Output("year-slider", "value"),
    Input("animate", "n_intervals"),
    State("year-slider", "value"),
    prevent_initial_call=True,
)
def update_figure(n, selected_year):
    index = years.index(selected_year)
    index = (index + 1) % len(years)
    year = years[index]
    return make_fig(year), year


@app.callback(
    Output("animate", "disabled"),
    Input("play", "n_clicks"),
    State("animate", "disabled"),
)
def toggle(n, playing):
    if n:
        return not playing
    return playing

app.run_server(host='0.0.0.0', port=8050, mode="jupyterlab", debug=True, threaded=True)